# Reciprocal Retriever
By combining the results from multiple retrievers, this retriever aims to improve the accuracy and relevance of search results. It employs reciprocal rank fusion, a technique that prioritizes results that consistently rank highly across different retrievers, ensuring that the most relevant information is presented to the user.

Pros:



- Prioritization of Consistent Results: By combining multiple retrievers and using reciprocal rank fusion, this method emphasizes results that consistently rank high across different approaches, potentially improving reliability.


Cons:



- Sensitivity to Retriever Performance: The effectiveness of this fusion technique depends on the quality and diversity of the individual retrievers being combined. This information is not from the sources, you may want to verify it independently.

In [1]:
from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama

ollama_embedding = OllamaEmbedding(
    model_name="mxbai-embed-large",
    base_url="http://localhost:11434",
)

ollama = Ollama(
    model="llama3.2:3b-instruct-fp16",
    base_url="http://localhost:11434"
)

Settings.llm = ollama
Settings.embed_model = ollama_embedding

In [2]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=["../documents/ods-cpp (1).pdf"]).load_data()

In [3]:
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=256)

index = VectorStoreIndex.from_documents(documents, transformations=[splitter])

In [4]:
from llama_index.retrievers.bm25 import BM25Retriever

vector_retriever = index.as_retriever(similarity_top_k=2)

bm25_retriever = BM25Retriever.from_defaults(
    docstore=index.docstore, similarity_top_k=2
)

/home/research/Faraz/Ollama_retrievers/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from llama_index.core.retrievers import QueryFusionRetriever

retriever = QueryFusionRetriever(
    [vector_retriever, bm25_retriever],
    similarity_top_k=2,
    num_queries=4,  # set this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,
    # query_gen_prompt="...",  # we could override the query generation prompt here
)

In [6]:
# apply nested async to run in a notebook
import nest_asyncio

nest_asyncio.apply()

In [7]:
nodes_with_scores = retriever.retrieve(
    "What is the difference between stack and queue?"
)

Generated queries:
Here are three search queries generated from your input:
1. "data structures stack vs queue"
2. "programming language concepts stack vs queue"
3. "algorithms stack vs queue differences"


In [8]:
for node in nodes_with_scores:
    print(f"Score: {node.score:.2f} - {node.text}...\n-----\n")

Score: 0.03 - The
structures in this chapter are made up of nodes that contain the list items.
Using references (pointers), the nodes are linked together into a sequence.
We ﬁrst study singly-linked lists, which can implement Stack and (FIFO)
Queue operations in constant time per operation and then move on to
doubly-linked lists, which can implement Deque operations in constant
time.
Linked lists have advantages and disadvantages when compared to
array-based implementations of the List interface. The primary disad-
vantage is that we lose the ability to access any element using get(i) or
set(i,x) in constant time. Instead, we have to walk through the list, one
element at a time, until we reach the ith element. The primary advantage
is that they are more dynamic: Given a reference to any list node u, we
can delete uor insert a node adjacent to uin constant time. This is true
no matter where uis in the list....
-----

Score: 0.03 - · · ·
remove ()/pop ()add (x)/push (x)
x
Figure 1.3: A s

In [9]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever)

In [12]:
response = query_engine.query("What is the difference between stack and queue?")


Generated queries:
Here are three search queries related to your input:
1. "data structure comparison stack vs queue"
2. "stack vs queue in computer science explanation"
3. "difference between LIFO and FIFO data structures"


In [13]:
from llama_index.core.response.notebook_utils import display_response

display_response(response)

**`Final Response:`** A Stack and a Queue are two different data structures that differ in how elements are added and removed.

In a Stack, elements are added to the top of the structure and removed from the top. This means that the most recently added element is also the first one to be removed. A Queue, on the other hand, is a First-In-First-Out (FIFO) data structure, where elements are added to the end and removed from the front.

This difference in ordering makes Stacks and Queues distinct and useful for different applications.